# Importing Libraries

In [1]:
import urllib.request as urllib2
import json
import time
import pandas as pd  
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split  

In [2]:
import warnings
warnings.filterwarnings("ignore")

# Thingspeak Configuration

In [3]:
READ_API_KEY= 'SZZOVTJU2AR48WK1'
CHANNEL_ID= '2868552'

In [4]:
# while True:
TS = urllib2.urlopen("http://api.thingspeak.com/channels/%s/feeds/last.json?api_key=%s" \
                       % (CHANNEL_ID,READ_API_KEY))

response = TS.read()
data=json.loads(response)


a = data['created_at']
b = data['field1']
c = data['field2']
d = data['field3']
e = data['field4']
f = data['field5']
g = data['field6']
print (a + "    " + b + "    " + c + "    " + d + "    " + e + "    " + f + "    " + g)
time.sleep(5)   

TS.close()

2025-04-03T06:25:03Z    28.97    45    0    9.82    32.75    -0.11


# Reading the Dataset

In [5]:
ds=pd.read_csv("/kaggle/input/weatheraus-csv/weatherAUS.csv")  

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/weatheraus-csv/weatherAUS.csv'

In [ ]:
import os

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


/kaggle/input/weatheraus-csv/weatherAUS.csv
/kaggle/input/feeds-csv/feeds (1).csv


In [ ]:
ds.head()    

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,...,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,No
1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,...,44.0,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,No
2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,...,38.0,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,No
3,2008-12-04,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,...,45.0,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No,No
4,2008-12-05,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,...,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,No


# Cleaning the Dataset

In [ ]:
ds = ds.drop(columns = ['Location','Evaporation', 'Sunshine', 'WindGustDir', 'WindGustSpeed', 'WindDir9am','WindDir3pm','WindSpeed9am', 'WindSpeed3pm', 'Cloud9am', 'Cloud3pm'])

In [ ]:
ds.head()

,Date,MinTemp,MaxTemp,Rainfall,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2008-12-01,13.4,22.9,0.6,71.0,22.0,1007.7,1007.1,16.9,21.8,No,No
1,2008-12-02,7.4,25.1,0.0,44.0,25.0,1010.6,1007.8,17.2,24.3,No,No
2,2008-12-03,12.9,25.7,0.0,38.0,30.0,1007.6,1008.7,21.0,23.2,No,No
3,2008-12-04,9.2,28.0,0.0,45.0,16.0,1017.6,1012.8,18.1,26.5,No,No
4,2008-12-05,17.5,32.3,1.0,82.0,33.0,1010.8,1006.0,17.8,29.7,No,No


In [ ]:
ds['Humidity'] = ds[['Humidity9am', 'Humidity3pm']].mean(axis=1)
ds['Pressure'] = ds[['Pressure9am', 'Pressure3pm']].mean(axis=1)
ds['Temperature'] = ds[['Temp9am', 'Temp3pm']].mean(axis=1)
ds = ds.drop(columns = ['Humidity9am', 'Humidity3pm','Pressure9am', 'Pressure3pm','Temp9am', 'Temp3pm'])
new_cols = ['Date','MinTemp', 'MaxTemp', 'Humidity', 'Pressure', 'Temperature', 'RainToday', 'RainTomorrow']
ds = ds.reindex(columns=new_cols)
ds.head()

,Date,MinTemp,MaxTemp,Humidity,Pressure,Temperature,RainToday,RainTomorrow
0,2008-12-01,13.4,22.9,46.5,1007.40,19.35,No,No
1,2008-12-02,7.4,25.1,34.5,1009.20,20.75,No,No
2,2008-12-03,12.9,25.7,34.0,1008.15,22.10,No,No
3,2008-12-04,9.2,28.0,30.5,1015.20,22.30,No,No
4,2008-12-05,17.5,32.3,57.5,1008.40,23.75,No,No


In [ ]:
ds.isnull().sum()

Date                0
MinTemp          1485
MaxTemp          1261
Humidity         1887
Pressure        14804
Temperature      1129
RainToday        3261
RainTomorrow     3267
dtype: int64

In [ ]:
ds = ds.dropna()

In [ ]:
ds.isnull().sum()

Date            0
MinTemp         0
MaxTemp         0
Humidity        0
Pressure        0
Temperature     0
RainToday       0
RainTomorrow    0
dtype: int64


# Splitting the Dependent and Independent Variables

In [ ]:
x = ds.iloc[:,1:7]
display(x.head())

,MinTemp,MaxTemp,Humidity,Pressure,Temperature,RainToday
0,13.4,22.9,46.5,1007.40,19.35,No
1,7.4,25.1,34.5,1009.20,20.75,No
2,12.9,25.7,34.0,1008.15,22.10,No
3,9.2,28.0,30.5,1015.20,22.30,No
4,17.5,32.3,57.5,1008.40,23.75,No


In [ ]:
y = ds.iloc[:,7]
display(y.head())

0    No
1    No
2    No
3    No
4    No
Name: RainTomorrow, dtype: object

# One Hot Encoding

In [ ]:
x_encoded = pd.get_dummies(x, columns=['RainToday'], drop_first=True)
x_encoded.rename(columns = {'RainToday_Yes':'RainToday'}, inplace = True)
x_encoded.head()

,MinTemp,MaxTemp,Humidity,Pressure,Temperature,RainToday
0,13.4,22.9,46.5,1007.40,19.35,False
1,7.4,25.1,34.5,1009.20,20.75,False
2,12.9,25.7,34.0,1008.15,22.10,False
3,9.2,28.0,30.5,1015.20,22.30,False
4,17.5,32.3,57.5,1008.40,23.75,False


In [ ]:
y_encoded = pd.get_dummies(y, columns=['RainTomorrow'], drop_first=True)
y_encoded.rename(columns = {'Yes':'RainTomorrow'}, inplace = True)
y_encoded.head()

,RainTomorrow
0,False
1,False
2,False
3,False
4,False


# Splitting the Train and Test Data

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x_encoded, y_encoded, test_size = 0.20, random_state=0)  

# Model Building

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model_rf = RandomForestClassifier()
model_rf.fit(x_train,y_train)

RandomForestClassifier()

# Predicting using Test Data

In [ ]:
pred = model_rf.predict(x_test)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, pred, normalize = True)

0.8325347483467311

# Predicting for the Actual Data from the Sensors

In [ ]:
import requests
import pandas as pd
import time
from datetime import datetime, timezone

CHANNEL_ID = "2868552"
READ_API_KEY = "SZZOVTJU2AR48WK1"

url = f"https://api.thingspeak.com/channels/{CHANNEL_ID}/feeds.json?api_key={READ_API_KEY}&results=5"

columns = ["Timestamp", "Temperature", "Humidity", "Rain Digital", "Rain Analog", "Gas Level", "Soil Moisture"]
df_live = pd.DataFrame(columns=columns)

while True:
    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()
        feeds = data.get("feeds", [])

        if not feeds:
            print("⚠️ No data received from ThingSpeak!")
            time.sleep(10)
            continue

        latest_entry = feeds[-1]  
        timestamp = latest_entry.get("created_at", None)

        if timestamp:
            timestamp_dt = datetime.strptime(timestamp, "%Y-%m-%dT%H:%M:%SZ").replace(tzinfo=timezone.utc)
            current_time = datetime.now(timezone.utc)
            time_difference = (current_time - timestamp_dt).total_seconds()

            print(f"🕒 Latest Data Timestamp: {timestamp_dt} (Difference: {time_difference} sec)")

            if time_difference <= 60:  # Check if data is within the last minute
                new_data = {
                    "Timestamp": timestamp,
                    "Temperature": latest_entry.get("field1", None),
                    "Humidity": latest_entry.get("field2", None),
                    "Rain Digital": latest_entry.get("field3", None),
                    "Rain Analog": latest_entry.get("field4", None),
                    "Gas Level": latest_entry.get("field5", None),
                    "Soil Moisture": latest_entry.get("field6", None)
                }

                df_live = pd.concat([df_live, pd.DataFrame([new_data])], ignore_index=True)
                print(f"✅ Data recorded: {new_data}")

                csv_path = "/kaggle/working/live_weather_data.csv"
                df_live.to_csv(csv_path, index=False)

                break
            else:
                print("⚠️ No recent data found in the last 1 minute, waiting...")

        else:
            print("❌ No timestamp found in latest data!")

    else:
        print(f"❌ Failed to fetch data! HTTP Status Code: {response.status_code}")

    time.sleep(10)  # Check for every 10 seconds


🕒 Latest Data Timestamp: 2025-04-03 06:09:46+00:00 (Difference: 22.329104 sec)
✅ Data recorded: {'Timestamp': '2025-04-03T06:09:46Z', 'Temperature': '27.99', 'Humidity': '58', 'Rain Digital': '0', 'Rain Analog': '9.80', 'Gas Level': '35.68', 'Soil Moisture': '-0.01'}


In [ ]:
df_live.head()  


,Timestamp,Temperature,Humidity,Rain Digital,Rain Analog,Gas Level,Soil Moisture
0,2025-04-03T06:09:46Z,27.99,58,0,9.80,35.68,-0.01


In [ ]:
df_live.head()


,Timestamp,Temperature,Humidity,Rain Digital,Rain Analog,Gas Level,Soil Moisture
0,2025-04-03T06:09:46Z,27.99,58,0,9.80,35.68,-0.01


In [ ]:
df_live.rename(columns = {'created_at':'Data and Time','entry_id':'Sr No.','field1':'Temperature', 'field2':'Humidity','field3':'Dew Point','field4':'Pressure','field5':'Altitude','field6':'Rainfall'},inplace = True)
df_live.head()

,Timestamp,Temperature,Humidity,Rain Digital,Rain Analog,Gas Level,Soil Moisture
0,2025-04-03T06:09:46Z,27.99,58,0,9.80,35.68,-0.01


In [ ]:
df_live[['Temperature', 'Humidity', 'Rain Analog', 'Gas Level', 'Soil Moisture']] = df_live[[
    'Temperature', 'Humidity', 'Rain Analog', 'Gas Level', 'Soil Moisture']].apply(pd.to_numeric, errors='coerce')

min_temp = df_live['Temperature'].min()
print('Min_Temp:', min_temp)

max_temp = df_live['Temperature'].max()
print('Max_Temp:', max_temp)


humidity = df_live['Humidity'].mean()
print('Humidity:', humidity)

pressure = df_live['Gas Level'].mean()
print('Pressure:', pressure)

temp = df_live['Temperature'].mean()
print('Temperature:', temp)

if df_live['Rain Analog'].mean() < 750:
    rain_today = 0
    print('Rainfall Today: No')
else:
    rain_today = 1
    print('Rainfall Today: Yes')

# Get the latest timestamp
prediction_date = df_live['Timestamp'].max()
print(f"Prediction Date: {prediction_date}")


Min_Temp: 27.99
Max_Temp: 27.99
Humidity: 58.0
Pressure: 35.68
Temperature: 27.99
Rainfall Today: No
Prediction Date: 2025-04-03T06:09:46Z


In [ ]:
print( df_live.columns.str.strip())  


Index(['Timestamp', 'Temperature', 'Humidity', 'Rain Digital', 'Rain Analog',
       'Gas Level', 'Soil Moisture'],
      dtype='object')


In [ ]:
print( df_live.columns)


Index(['Timestamp', 'Temperature', 'Humidity', 'Rain Digital', 'Rain Analog',
       'Gas Level', 'Soil Moisture'],
      dtype='object')


In [ ]:
print("project ended")

project ended


In [ ]:
import numpy as np
import requests

# # Modify test data to simulate rainy conditions
# min_temp = 18  # Lower temperature
# max_temp = 22  # Lower temperature
# humidity = 90  # High humidity

# pressure = 990  # Low pressure
# temp = 20  # Moderate temperature
# rain_today = 1  # Indicates it rained today


test_data = np.array([[min_temp, max_temp, humidity, pressure, temp, rain_today]])

# Make prediction
rain_predict = model_rf.predict(test_data)[0]  # first element from the array

# Print prediction
if rain_predict == 1:
    print("Yes, it will rain tomorrow")
else:
    print("No, it won't rain tomorrow")

TELEGRAM_BOT_TOKEN = "7793358427:AAEshxCzFy_qTbAXx3LawZRlFWFiNVA5mPQ"

TELEGRAM_CHAT_ID = "1814731517"

# Function to send message
def send_telegram_message(message):
    url = f"https://api.telegram.org/bot{TELEGRAM_BOT_TOKEN}/sendMessage"
    payload = {
        "chat_id": TELEGRAM_CHAT_ID,
        "text": message
    }
    response = requests.post(url, json=payload)  
    return response.json()

# Define prediction text
if rain_predict == 5:
    prediction_text = "☀️ Prediction: No rain tomorrow!"
else:
    prediction_text = "🌧️ Prediction: It will rain tomorrow!"
send_telegram_message(prediction_text)


No, it won't rain tomorrow


{'ok': True,
 'result': {'message_id': 37,
  'from': {'id': 7793358427,
   'is_bot': True,
   'first_name': 'Rain☔ alert',
   'username': 'RAIN_ALERTING_BOT'},
  'chat': {'id': 1814731517, 'first_name': 'Venkat', 'type': 'private'},
  'date': 1743660608,
  'text': '🌧️ Prediction: It will rain tomorrow!'}}

In [ ]:
print("project ended with telegram alert")

project ended with telegram alert
